#### 코노가자  
---------
# 칵테일 왓챠!  
평소 취향을 통해서 자신에게 맞는 칵테일을 추천해주는 시스템을 만드는 프로젝트이다.  
입력은 재료, 주량(알코올 도수), 탄산 여부로 받으며, 출력은 5개 내외의 칵테일을 추천해준다.

---------
# 프로젝트 과정  

## 0. 주제 선정
다양한 아이디어를 회의를 통해 추려서 선정하였다.

## 1. 데이터 수집(크롤링)

### 1) 모든 칵테일 정보 크롤링

https://www.thecocktaildb.com/에서 모든 종류의 칵테일 이름과 베이스를 포함한 재료들을 크롤링하였다. 
재료를 크롤링하는 과정에 있어서 예외사항들을 다음과 같이 처리하였다.

- 개수/단위 안나와 있는 것들은 통으로 재료명으로 넣었음
- 개수가 소수점이나 분수로 나와있는 것들은 다 소수점으로 통일
- Juice of~/twist of~ 은 단위가 아닌 이름에 포함시켰고, 단위는 따로 빼서 계산했음 (소수점, 분수 포함)
- "2-3 Lime" 이런식으로 개수가 범위지정 되어 있는 것들은 평균화했음
-  수정 예정인 예외들 : "2 or 3 ice" 같은 경우도 평균처리하기, dash-dashes는 dash로 통일하기, 단위 "oz" 앞뒤 띄어쓰기 지우기,

1) 개수를 소수점 둘째자리까지 반올림 (1.66666… 와 같은 경우를 위해)  
2) 10-12 oz coca-cola와 같이 개수가 두자리수 이상인 경우를 위한 코드 추가. 소수점, 분수 등에도 마찬가지.   
3) 단위 형태 통일 및 정리  
    - 복수형->단수형 통일 : parts, cups, dashes, cubes, wedges, sticks, splashes, pieces, pinches… 등 복수형 단위들 뒤의 es/s를 지워줌. packages/bottles 같은 경우는 당연히 s만 지움.
    - tablespoons/tbsp/spoons -> tblsp로 통일
    - quart -> qt로 통일
    - 대소문자를 기준으로 단위/재료명을 나누었더니 large/frozen/crushed/cracked .. 등과 같이 재료명에 해당하는 형용사들이 단위로 분류된 경우가 있어 이러한 경우들을 재료명에 병합. 비슷한 이유로 리터를 뜻하는 “L”이 재료명으로 분류된 것을 확인해 L은 단위로 분류하도록 코딩. 
    - 2 measures tequilla : 데킬라 한잔에 2oz -> 4 oz tequilla로 바꿔 넣음  
4) 너무 예외적인 경우들은 그냥 단일 경우를 위한 if 문을 생성해 제어함. ex) “70ml/2fl oz Vodka”(slash 앞뒤로 바로 숫자가 붙지 않고 단위가 붙어온 경우), “7-up”(dash 앞에는 숫자, 뒤에는 문자가 온 경우), “, orange Carbonated”(쉼표로 시작하는 경우. 아마 웹상 오타), “4cl Vodka”(개수와 단위 사이에 공백이 없는 경우) 등  

### 2) 유명한 칵테일 정보 크롤링


-
## 2. 데이터 전처리 (보완)

- 단위 전부 oz로 통일  
    **<재료 단위>**  
    splash = dash = 0.03125 oz
    scoop = 1 oz
    can = 7.5 oz
    sprig = inch = gr(gram) = 0.03 oz
    kg = 33 oz
    lb = 15 oz
    juice of ~ = 5 tblsp = 2 oz
    cube = piece = 0.06 oz = sugar
    chunk = 5 oz
    slice = wedge = 0.2 oz
    package = 6 oz
    stick = 0.06 oz
    pinch = 0.01 oz
    whole egg = 1.5 oz
    egg white = 0.9 oz
    egg yellow = 0.6 oz
    cherry = 0.3 oz
    lemon = 4.5 oz
    banana = 4 oz
    papaya = mango = 9 oz
    apple = 7 oz
    apricot = 1 oz
    kiwi = 3 oz
    syrup = 0.0375 oz
    wormwood = vanilla = cloves = 0.03 oz  
    **<베이스 단위>**  
    oz = 약 30ml
    tsp = 0.125 oz
    tblsp = 0.375 oz
    dash = 0.03125 oz
    dr  0.00667 oz
    ml = 0.03 oz
    cl = 0.3 oz
    dl = 3.3 oz
    L = 33 oz
    cup = 8 oz
    pint = 16 oz
    shot = 3.5 oz
    part = 1 oz
    jigger/bar = glass = 0.3 oz  
    bottle = 25 oz
    wine glass = 34 oz
    gal = 128 oz
    qt = 32 oz
    fifth = 25.6 oz
    tenth = 12 oz
    split = 6 oz
    proof X

## 3. 데이터 시각화 Feature Engineering & Dataset 구성하기

- 베이스랑 재료 구분
재료구분 열을 따로 만들어서 구분해놓는다. 기본재료는 Ingridients로, 베이스는 베이스의 종류대로 적어준다. 나중에 Liqueur를 좀 더 세세하게 처리하고, 알코올 도수를 처리하기 위함이다.

- 나중 태그 추천을 위해서 재료비율 열과 알코올 도수 열을 만든다.
-> 재료 비율을 유사도에서 사용하고자 했으나 *그룹화*를 선택하는 방식이 되면서 *재료의 비율*보다는 들어갔는지 *재료 여부*가 더 중요해졌기 때문에 사용하지 않게 되었다. 

- 알코올 도수
각각 베이스에 맞는 알코올 도수를 입력해주는 ABV열을 만들었다.
이를 바탕으로 칵테일 하나당 알코올 도수를 식을 사용하여 각 칵테일의 마지막 행에 alcohol 열을 만들어서 추가해주었다.
알코올 도수 분포를 데이터 시각화를 통해 알아보고, describe()를 통해 얻은 3분위 값으로 기준을 나누어서 알코올을 분류해주었다. 이를 alcohol_classification 열로 구분해놓았다.

도수 0 1 2 3 단계  
0<= ... <6 : 0, 6<= <15 : 1, 15<= <25: 2, 25<= : 3

- 재료 구분
ingrid_classification 열은 각 재료의 특성을 파악하는 열이다.
재료이름에 들어간 단어들을 모두 추리고 빈도수를 파악하여 크게 영향끼치지 않을 재료들을 제외하고 다음과 같은 기준으로 각각 단어들을 분류하였다.
이것은 재료 구분열의 Ingridients와 Liqueur를 한정으로 하였다.

재료 0 1 2 3 4  
hot: 0, 리프레싱: 1, 프루트: 2, 스위트: 3, 스무디: 4, dry: 5, 탄산: 6

이 재료구분 열을 groupby를 사용하여 그룹화하여 칵테일을 크게 분류해줄 수 있다.

## 4. 모델링과 학습 (분류)

**<칵테일 유사성 분류 계획>**  
1) 들어간 재료의 종류에 따라서 Decision Tree로 유사성이 높은 후보를 5~7개 정도 추리기  
   : 베이스, 재료 각각에 가중치 부여해서 유사성 비교  
   : 베이스 / 재료 / 베이스+재료로 비교해서 가장 유사성 높은 것 선택  
2) 후보들을 들어간 재료의 비율에 따라 HeatMap으로 그려서 유사성 분류  
3) Test set을 어떻게 할지 고민중…  
   : 그래프에서 거리를 대충 따올 수 있으면 그거랑 비교하기  

**계획**은 이랬으나, 근본적으로 추천시스템에는 target_data라는 것을 구분하기가 어려워서, test set이 존재하기 어려웠고, 그로 인해서 decision tree도 배운 선에서 구현할 수 없었다. HeatMap 또한 연속적인 숫자의 데이터, 그리고 2차원의 데이터가 필요했는데,
우리가 고려하는 부분과는(알코올로 분류, 재료로 분류,, 등) 달라서 heatmap으로 구현할 수 없다는 것을 깨달았다.

대안의 방법으로 **Groupby**를 알아보았다. 칵테일을 추천함에 있어서 가장 영향을 많이 끼치는 것은 맛이고, 재료가 맛을 결정한다고 생각하여 재료를 구분했던 대로 모든 칵테일을 구분화 할 수 있었다. 그리고 각 구분에 해당하는 것은 list로 저장되어있었기 때문에 이 list를 읽어서 칵테일 이름을 빼올 수 있었다. 이 list에는 다양한 칵테일이 존재하기 때문에, 다시 한 그룹에 해당하는 칵테일을 어떻게 나눌 것인가에 대한 생각을 하였다. 그 결과, 칵테일을 잘 모르는 사람들에게는 보다 대중적인 칵테일이 나을 것이라고 생각하였다. 이를 구현하기 위해 칵테일을 검색했을 때 나오는 정보들을 크롤링하여, 칵테일마다 빈도수를 구하여 대중성을 파악하였고, 칵테일을 대중적인 순서로 출력되도록 하였다.

이 이후에, 알코올 도수를 제어하는 if문을 넣어 해당 칵테일들이 사용자가 입력한 알코올 범위에 있도록 분류하였다.